In [1]:
# Standard Imports
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
# Performance
from time import time

In [3]:
# Machine Learning
from sklearn.linear_model import LogisticRegression

In [4]:
# Helper
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import accuracy_score

In [5]:
# Load the Data
train = pd.read_csv('input/train2.csv')
test = pd.read_csv('input/test2.csv')

# Logistic regression

*penalty* = 'l2' (specify the norm used in penalization)

*C* = 1.0 (inverse of regularization strength; smaller values specify stronger regularization)

*random_state* = None (seed of the pseudo random number generator to use when shuffling the data)

In [12]:
# Predictors
predictors = ["Pclass", "Gender", "AgeFill", "Port", "Fare"]

In [13]:
# Initialize our algorithm
alg = LogisticRegression()

# Train the algorithm using all the training data
start = time()
alg.fit(train[predictors], train["Survived"])
print("LogisticReg Training finished in %.2f s" % (time() - start))

LogisticReg Training finished in 0.06 s


In [14]:
# Basic Evaluation on Training Set
start = time()
train_pred = cross_val_predict(alg, train[predictors], train["Survived"], cv=2)
print("LogisticReg Evaluation finished in %.2f s" % (time() - start))

LogisticReg Evaluation finished in 0.02 s


In [15]:
# Display Accuracy
print("Accuracy: ", accuracy_score(train["Survived"], train_pred))

Accuracy:  0.784026996625


In [ ]:
# Make predictions using the test set.
predictions = alg.predict(test[predictors])

In [ ]:
# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": predictions
    })

# Any files you save will be available in the output tab below
submission.to_csv('submission.csv', index=False)